# main_baseline_new
a unit test, which pull the first observation for each person for each non-missing temporal feature
and concatenate these to the static features and the corresponding mask matrix. To check whether the 
predictor is useful.

In [71]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score
import pandas as pd
pd.set_option('display.max_columns', None)

In [29]:
org_data = np.load("./MIMIC_timeseries/24hours/series/imputed-normed-ep_1_24.npz", allow_pickle=True)
temporal_feature_dim = 12
static_feature_dim = 5
targets_dim = 21
adm_num = len(org_data['ep_tdata'])
temporal_feature = np.zeros((adm_num, temporal_feature_dim))
temporal_mask = np.zeros((adm_num, temporal_feature_dim))
static_feature = np.zeros((adm_num, static_feature_dim))
targets = np.zeros((adm_num, targets_dim))


In [30]:
for i in range(adm_num):
    for j in range(temporal_feature_dim):
        temp_f = org_data["ep_tdata"][i][:, j]
        temp_m = org_data["ep_tdata_masking"][i][:, j]
        if any(temp_m):
            temporal_feature[i, j] = temp_f[temp_m == True][0]
            temporal_mask[i, j] = 1
        else:
            temporal_feature[i, j] = 0
            temporal_mask[i, j] = 0
        

In [40]:
static_feature = org_data["adm_features_all"]
y_icd9 = org_data["y_icd9"]
y_mor = org_data["y_mor"]
targets = np.concatenate((y_icd9, y_mor), 1)
predictors = np.concatenate((static_feature, temporal_feature, temporal_mask), 1)
X_train, X_test, y_train, y_test = train_test_split(predictors, targets, test_size=0.33, random_state=42)

In [45]:
clf_models_list = []
for i in range(targets_dim):
    clf_models_list.append(LogisticRegression(max_iter=1000).fit(X_train, y_train[:, i]))



/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

In [81]:
auprc_lists = np.zeros(targets_dim)
auroc = np.zeros(targets_dim)

for i in range(targets_dim):
    cur_y_true = y_test[:, i]
    cur_y_score = clf_models_list[i].predict_proba(X_test)[:,1] 
    # y_score used to compute auprc/auroc is the probability estimates of the positive class
    # through "clf_models_list[i].classes_" we know the order for each target class's label is [0 1]
    auprc_lists[i] = average_precision_score(cur_y_true, cur_y_score)
    auroc[i] = roc_auc_score(cur_y_true, cur_y_score)

In [73]:
auprc_lists

array([0.44669215, 0.23952274, 0.77001515, 0.47824311, 0.37871609,
       0.32696577, 0.94060067, 0.6018682 , 0.59582595, 0.70220011,
       0.05419829, 0.14603482, 0.23969321, 0.08384789, 0.41713585,
       0.10544544, 0.04146463, 0.50384245, 0.5575825 , 0.36984488,
       0.23998469])

In [74]:
baseline_for_auprc = np.zeros(21)
for i in range(targets_dim):
    baseline_for_auprc[i] = np.count_nonzero(y_test[:,i]) / len(y_test[:, i])
baseline_for_auprc

array([0.25238176, 0.17106159, 0.67786662, 0.36415447, 0.31753998,
       0.28317455, 0.82332426, 0.48205172, 0.38618578, 0.38618578,
       0.00425315, 0.10079959, 0.18747873, 0.0358115 , 0.31413746,
       0.08225587, 0.02943178, 0.44683566, 0.46444369, 0.33370194,
       0.10037428])

In [75]:
col_names = []
for i in range(1, 22):
    col_names.append("t"+str(i))
auprc_compares = np.zeros((2, targets_dim))
auprc_compares[0] = baseline_for_auprc
auprc_compares[1] = auprc_lists
auprc = pd.DataFrame(auprc_compares, columns=col_names, index = ['base', "logistic_model"])
auprc = auprc.round(3)
auprc

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
base,0.252,0.171,0.678,0.364,0.318,0.283,0.823,0.482,0.386,0.386,0.004,0.101,0.187,0.036,0.314,0.082,0.029,0.447,0.464,0.334,0.10
logistic_model,0.447,0.240,0.770,0.478,0.379,0.327,0.941,0.602,0.596,0.702,0.054,0.146,0.240,0.084,0.417,0.105,0.041,0.504,0.558,0.370,0.24


In [82]:
auroc_df = pd.DataFrame(auroc.reshape(1, targets_dim), columns=col_names)
auroc_df = auroc_df.round(3)
auroc_df

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
0,0.694,0.609,0.633,0.635,0.576,0.561,0.806,0.63,0.663,0.76,0.94,0.613,0.586,0.686,0.61,0.57,0.619,0.561,0.613,0.548,0.702
